In [1]:
import pandas as pd
import numpy as np

In [303]:
from DecisionTree_recursive import DecisionTree
dt = DecisionTree(max_depth = 20, min_node = 20, min_leaf=10, threshold = 1e-3)
dt.build_tree(train_data, 20, get_gini(train_data))

NameError: name 'n' is not defined

## Read

In [217]:
## read train data
data = pd.read_table('training.txt', sep=' ', header=None)
# name label
data.rename(columns={0:'label'}, inplace=True)
# sepearte labels
label = data.iloc[:, 0]

# remove index: from data
data = data.iloc[:, 1:].applymap(lambda x: x[x.find(':')+1:])
data = data.astype(int)

# rename columns and merge label
data.columns = list(range(data.shape[1]))
data = data.join(label)

In [221]:
## read test data
test = pd.read_table('testing.txt', sep=' ', header=None)

# remove index: from data
test = test.applymap(lambda x: x[x.find(':')+1:])
test = test.astype(int)

In [62]:
# train test split
n = data.shape[0]
idx = np.random.randint(0, n, int(n/20))
val_data = data.iloc[idx,:]
train_data = data.drop(idx)

# data indexing
train_data.index = range(0, train_data.shape[0])
val_data.index = range(0, val_data.shape[0])

## Decision Tree

In [342]:
# Build tree
def build_tree(data, depth, min_node, min_leaf, base_gini):
    threshold = 3e-3
    
    if depth == 0 or data.shape[0] < min_node:
        pass
    else:
        column, feature, min_gini = find_best_split(data, min_leaf)
        if not min_gini:
            pass
        elif base_gini-min_gini > threshold:
            left_idx, right_idx = data_split(data, column, feature)
            left_label = get_label(data.loc[left_idx, :])
            right_label = get_label(data.loc[right_idx, :])

            tree = {}
            tree[(column, feature, 'left', len(left_idx), left_label)] = build_tree(data.loc[left_idx, :], depth-1, min_node, min_leaf, min_gini)
            tree[(column, feature, 'right', len(right_idx), right_label)] = build_tree(data.loc[right_idx, :], depth-1, min_node, min_leaf, min_gini)
            
            return tree
        else:
            pass


In [310]:
## Predict
def predict(data, tree):
    train = data.copy()
    tree_list = [(train ,tree)]
    while tree_list:
        train, tree_dict = tree_list.pop(0)
        
        for key, item in tree_dict.items():
            if item is not None:
                column, feature, side, number, label = key
                #print(key)

                # on left leaf
                if side == 'left':
                    left_idx, right_idx = data_split(train, column, feature)
                    data.loc[left_idx, 'predict'] = label
                    tree_list.append((data.loc[left_idx, :],item))

                # on right leaf
                else:
                    left_idx, right_idx = data_split(train, column, feature)
                    data.loc[right_idx, 'predict'] = label
                    # append leaf
                    tree_list.append((data.loc[right_idx, :], item))

        
    return 0

In [311]:
def get_gini(data):
    _, counts = np.unique(np.array(data.label), return_counts= True)
    
    return 1-np.sum(np.square(counts/data.shape[0]))

In [312]:
def data_split(data, column, feature):
    
    left = data[column][data[column]==feature].index.values
    right = data[column][data[column]!=feature].index.values
    
    return left, right

In [313]:
def get_label(data):
    label_count = data.groupby('label').label.count()
    
    return label_count.idxmax()

In [332]:
def find_best_split(data, min_leaf):
    subset_n = int(np.sqrt(data.shape[1])+1)
    # exclude 'label' column
    columns = np.random.choice(data.columns[:-1], size = subset_n, replace=False)
    gini_list = []
    # loop over columns except labels
    for column in data.loc[:,columns]:
        if column != 'label':
            for feature in np.unique(data[column]):
                left_idx, right_idx = data_split(data, column, feature)

                # check if splits are smaller than min_leaf
                left_n = len(left_idx)
                right_n = len(right_idx)
                if left_n <= min_leaf and right_n <= min_leaf:
                    pass
                elif left_n > min_leaf and right_n > min_leaf:
                    left_ratio = left_n/n
                    right_ratio = right_n/n
                    gini = left_ratio*get_gini(data.loc[left_idx,:])+right_ratio*get_gini(data.loc[right_idx, :])
                    gini_list.append((column, feature, gini))

    if not gini_list:
        return None, None, None
    else:
        return min(gini_list, key= lambda x: x[2])

In [343]:
%%time
tree = build_tree(train_data, depth = 15, min_node = 10, min_leaf = 2, base_gini = get_gini(data))

Wall time: 43.4 s


In [344]:
train_data.predict = None
pred = predict(train_data, tree)
print(np.unique(train_data.predict, return_counts=True))
print(np.mean(train_data.label==train_data.predict))
pred = predict(val_data, tree)
print(np.mean(val_data.label==val_data.predict))

(array([1, 2, 3, 4], dtype=int64), array([659, 723, 821, 653], dtype=int64))
0.5826330532212886
0.44666666666666666


## Test set

In [345]:
%%time
tree = build_tree(data, depth = 15, min_node = 10, min_leaf = 2, base_gini = get_gini(data))

Wall time: 46.2 s


In [346]:
pred = predict(test, tree)

## Save

In [347]:
with open('result.txt', 'w') as fp:
    [fp.write(str(int(predict))+'\n') for predict in test.predict]